In [1]:
from main import *

In [2]:
# for 2nd tranfer - inner cluster
def inner_cluster_2nd(cluster_no, hosp_ind, pat_type, df_group: pd.DataFrame):
    M = 10**10
    ind = 10**10
    tmp_cluster = df_group.get_group(cluster_no)
    
    if any([pat_type == 0, pat_type == 3]):
        hosp_list = [i for i in tmp_cluster[tmp_cluster['level'] > 0].index]
    elif any([pat_type == 6, pat_type == 7]):
        hosp_list = [i for i in tmp_cluster[tmp_cluster['level'] == 2].index]
    # print(hosp_list)
    
    if (len(hosp_list) == 0):
        return -1, -1
    else:
        for i in range(len(hosp_list)):
            cal_time_ = dis_mat.at[hosp_ind, hosp_list[i]]
            cal_time = cal_time_ + home.at[hosp_list[i],'admin_time'] + home.at[hosp_list[i],'test_time'] + home.at[hosp_list[i], 'treat_time']
            if(cal_time < M):
                M = cal_time
                ind = hosp_list[i]
        return ind, M
# for 2nd tranfer - intra cluster
def intra_transfer(hosp_ind, pat_type, arr):
    M = 10**10
    ind = 10**10

    if any([pat_type == 0, pat_type == 3]):
        hosp_list = [i for i in home[home['level'] > 0].index]
    elif any([pat_type == 6, pat_type == 7]):
        hosp_list = [i for i in home[home['level'] == 2].index]
    
    for i in range(len(hosp_list)):
        cal_time_ = dis_mat.at[hosp_ind, hosp_list[i]]
        cal_time = cal_time_ + home.at[hosp_list[i],'admin_time'] + home.at[hosp_list[i], 'test_time'] + home.at[hosp_list[i], 'treat_time']
        if(cal_time < M):
            M = cal_time
            ind = hosp_list[i]
    cluster_no, hosp_index = find_hosp_index(ind, arr)
    return cluster_no, hosp_index, M

In [3]:
eval_dict_list = {
    'Data1': {
        'N1': {'TD1': [], 'NCP1': [], 'O451': []}
    },
    'Data2': {
        'N1': {'TD1': [], 'NCP1': [], 'O451': []}
    },
    'Data3': {
        'N1': {'TD1': [], 'NCP1': [], 'O451': []}
    }
}

In [4]:
for sheet in range(1, 11):
    for d_type in range(1, 4):
        p_data = read_data_by_sheet(d_type, sheet)
        # patient data generator
        pat_data_info = p_data
        # patient index
        patient_index = [i for i in pat_data_info.index]

        cluster_number = 1
        # for cluster_number in range(1,5):
        # grouping hospitals by using kmeans method
        home['cluster'] = cluster_number
        home.at[14, 'cluster'] = cluster_number + 1
        KM = KMedoids(n_clusters = cluster_number, init = 'k-medoids++', random_state = 5)
        home.iloc[:14]['cluster'] = KM.fit_predict(home.iloc[:14][['lat','long']])
        hosp_clus = home.groupby('cluster')
        hosp_clus_list = hosp_clus.count().index

        # hospital index array based on cluster
        cluster_hosp_index = [[] for i in range(len(hosp_clus_list))]
        for i in range(len(hosp_clus_list)):
            cluster_hosp_index[i] = [h for h in hosp_clus.get_group(i).index]

        hosp_loc = [[] for i in range(len(hosp_clus_list))]
        for i in hosp_clus_list:
            hosp_loc[i] = [[] for h in range(len(hosp_clus.get_group(i)))]
        
        # simulation
        for i in patient_index[:]:
            cal_len = 0
            _1st_, _1st_len = inner_cluster_1st(i, pat_data_info)
            if (_1st_ > 14):
                continue
            else:
                select_cluster = [i for i in hosp_clus.get_group(home.at[_1st_, 'cluster']).index]
                tmp_hosp_ind = select_cluster.index(_1st_)
                trans_move = patient_trans(i, _1st_, pat_data_info)
                
                if any([trans_move == 1, trans_move == 2, trans_move == 4, trans_move == 5, trans_move == 8]):
                    cal_len = cal_len + _1st_len
                    hosp_loc[home.at[_1st_, 'cluster']][tmp_hosp_ind].append(cal_len)
                    
                else:
                    _2nd_, _2nd_len = inner_cluster_2nd(home.at[_1st_, 'cluster'], _1st_, trans_move, hosp_clus)
                    if (_2nd_ == -1):
                        intra_cluster, intral_hosp, intra_dis = intra_transfer(_1st_, trans_move, cluster_hosp_index)
                        cal_len = cal_len + intra_dis + _1st_len
                        hosp_loc[intra_cluster][intral_hosp].append(cal_len)
                        
                    else:
                        cal_len = cal_len + _2nd_len + _1st_len
                        hosp_loc[home.at[_1st_, 'cluster']][select_cluster.index(_2nd_)].append(cal_len)
        sum_time = 0
        sum_len = 0
        for row in range(len(hosp_loc)):
            for col in hosp_loc[row]:
                sum_len += len(col)
                sum_time += sum(col)
        eval_dict_list[f'Data{d_type}'][f'N{cluster_number}'][f'NCP{cluster_number}'].append(sum_len)
        eval_dict_list[f'Data{d_type}'][f'N{cluster_number}'][f'TD{cluster_number}'].append(sum_time/sum_len)
        
        pass_pat = 0
        for i in range(len(hosp_loc)):
            for j in range(len(hosp_loc[i])):
                for h in hosp_loc[i][j]:
                    if h > 4.5*60*60:
                        pass_pat += 1
                    else:
                        continue
        eval_dict_list[f'Data{d_type}'][f'N{cluster_number}'][f'O45{cluster_number}'].append(pass_pat)

In [5]:
result_excel = pd.DataFrame()
for i in ['Data1', 'Data2', 'Data3']:
    for j in ['TD1', 'NCP1', 'O451']:
        result_excel.at[i, j] = np.mean(eval_dict_list[i]['N1'][j])

with pd.ExcelWriter('exp_results_with_tor.xlsx', engine="openpyxl", mode="a") as writer:
    result_excel.to_excel(writer, sheet_name='S_cur')